<a href="https://colab.research.google.com/github/MASHAR27/BCHACKS2024/blob/main/New_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget file.weasoft.com/trans2.seq

--2024-01-28 00:48:35--  http://file.weasoft.com/trans2.seq
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66819385 (64M)
Saving to: ‘trans2.seq’

trans2.seq          100%[===================>]  63.72M   148MB/s    in 0.4s    

2024-01-28 00:48:36 (148 MB/s) - ‘trans2.seq’ saved [66819385/66819385]



In [2]:
import torch
import numpy as np

# !pip install wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'

DIM = 512
import math
import torch.nn as nn
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)].to(device)
        return x
#https://pytorch.org/tutorials/beginner/transformer_tutorial.html

class Music(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.pos = PositionalEncoding(DIM, max_len=201)#torch.nn.Embedding(200, DIM).to(device)
    self.projection = torch.nn.Embedding(128, DIM).to(device)
    self.trans = torch.nn.TransformerEncoder(torch.nn.TransformerEncoderLayer(DIM, 16, batch_first=True), 16).to(device)
    self.output = torch.nn.Linear(DIM, 128).to(device)
  def forward(self, x):
    #pos = self.pos(torch.arange(0,200).to(device))
    feature = self.projection(x)
    feature = self.pos(feature)
    feature = self.trans(feature, mask=torch.nn.Transformer.generate_square_subsequent_mask(200), is_causal=True)
    return self.output(feature)
model = Music()
model(torch.rand(1,200).to(torch.int32).to(device)).shape


torch.Size([1, 200, 128])

In [ ]:
with open("trans2.seq","r") as f:
  data=[[int(j) for j in i.split(" ")[:200] if j!=""] for i in f.read().split("\n") if len(i.split(" "))>=200]

new_data = []
for i in data:
  if len(i)==200:
    new_data.append(i)


In [ ]:
new_data = np.array(new_data)

In [ ]:
new_data.shape

In [ ]:

import pylab
pylab.hist((new_data).flatten())

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

run = wandb.init(
    project="ba-hacks",
)

In [ ]:
BATCH_SIZE = 32

import random
def getdata():
  data = random.choices(new_data, k=BATCH_SIZE)
  data = np.array(data)
  return np.concatenate((np.zeros((BATCH_SIZE,1)),data[:,:-1]), axis=1), data

np.concatenate((np.zeros((10,1)),new_data[:10,:-1]), axis=1).shape

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.5)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler =  torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, len(new_data)//BATCH_SIZE*10)

In [ ]:
model = model.to(device)
runing_loss = 0
count = 0
for epoch in range(30):
  for i in range(len(new_data)//BATCH_SIZE):
    x, y = getdata()
    x, y = torch.tensor(x).to(device).to(torch.int32), torch.tensor(y).to(device).to(torch.float32)
    output = model(x)
    loss = loss_fn(output, y)
    loss.backward()
    runing_loss += loss.item()
    count += 1
    if count==50:
      wandb.log({"loss":runing_loss/50})
      runing_loss = 0
      count = 0
    optimizer.step()
    scheduler.step()
torch.save(model, "music.mod")